In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import numpy as np
import pandas as pd
import re
# from rake_nltk import Rake

import numpy as np

from sklearn.metrics.pairwise import cosine_similarity

from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
train=pd.read_csv('train_feature/train2yu_100.csv')

In [ ]:
train.shape

In [ ]:
test=pd.read_csv('train_feature/test2yu_100.csv')

In [ ]:
test.shape

In [ ]:
def get_paper(x):
    jj=x['journal']
    text=x['description_text']
    if jj in text:
        return 1
    return 0
def get_year(x):
    try:
        
        jj=str(int(x['year']))
        text=x['description_text']
        if jj in text:
            return 1
        return 0
    except:
        return x
def get_year_int(x):
    try:
        return int(x)
    except:
        return x

In [ ]:
data_p=pd.concat([train,test]).reset_index(drop=True)

In [ ]:
data_p['is_have_paper']=data_p.apply(get_paper,1)

In [ ]:
# data_p['is_have_year']=data_p.apply(get_year,1)

In [ ]:
data_p['year']=data_p['year'].apply(get_year_int)

In [ ]:
def get_tf_sim(train_query_tf,train_title_tf):
    # 余弦
    v_num = np.array(train_query_tf.multiply(train_title_tf).sum(axis=1))[:, 0]
    v_den = np.array(np.sqrt(train_query_tf.multiply(train_query_tf).sum(axis=1)))[:, 0] * np.array(
            np.sqrt(train_title_tf.multiply(train_title_tf).sum(axis=1)))[:, 0]
    v_num[np.where(v_den == 0)] = 1
    v_den[np.where(v_den == 0)] = 1
    v_score1 = 1 - v_num / v_den

    # 欧式
    v_score = train_query_tf - train_title_tf
    v_score2 = np.sqrt(np.array(v_score.multiply(v_score).sum(axis=1))[:, 0])

    # 曼哈顿
    v_score = np.abs(train_query_tf - train_title_tf)
    v_score3 = v_score.sum(axis=1)

    return  v_score1,v_score2,v_score3

In [ ]:
import gc
gc.collect()

In [ ]:
article = list(data_p['key_text_pre'].fillna(''))
article.extend(list(data_p['title_pro'].fillna('')))
article.extend(list(data_p['abstract_pre'].fillna('')))
article.extend(list(data_p['description_text'].fillna('')))
article.extend(list(data_p['title'].fillna('')))
article.extend(list(data_p['abstract'].fillna('')))
article=list(set(article))

In [ ]:
tft = TfidfVectorizer(ngram_range=(1,1), min_df=2, max_df=0.5, sublinear_tf=True)
tft.fit(article)

In [ ]:
train_query_tf = tft.transform(data_p['key_text_pre'].fillna('').values)
train_query_tf2 = tft.transform(data_p['description_text'].fillna('').values)
train_title_tf = tft.transform(data_p['title_pro'].fillna('').values)
train_title_tf2 = tft.transform(data_p['abstract_pre'].fillna('').values)
train_title_tf3 = tft.transform(data_p['title'].fillna('').values)
train_title_tf4 = tft.transform(data_p['abstract'].fillna('').values)

In [ ]:
import gc
data_p['tfidf_cos'],data_p['tfidf_os'] ,data_p['tfidf_mhd']=get_tf_sim(train_query_tf,train_title_tf)
print(0)
data_p['tfidf_cos2'],data_p['tfidf_os2'] ,data_p['tfidf_mhd2']=get_tf_sim(train_query_tf,train_title_tf2)
print(1)
data_p['tfidf_cos_2'],data_p['tfidf_os_2'] ,data_p['tfidf_mhd_2']=get_tf_sim(train_query_tf2,train_title_tf)
print(2)
data_p['tfidf_cos2_2'],data_p['tfidf_os2_2'] ,data_p['tfidf_mhd2_2']=get_tf_sim(train_query_tf2,train_title_tf2)
print(3)
data_p['tfidf_cos2_3'],data_p['tfidf_os2_3'] ,data_p['tfidf_mhd2_3']=get_tf_sim(train_query_tf2,train_title_tf3)
print(4)
data_p['tfidf_cos2_4'],data_p['tfidf_os2_4'] ,data_p['tfidf_mhd2_4']=get_tf_sim(train_query_tf2,train_title_tf4)
print(5)
del  train_title_tf,train_query_tf,train_query_tf2,train_title_tf2,train_title_tf3,train_title_tf4
gc.collect()

In [ ]:
import time
import  numpy as np
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
import re


def tokenize(sentence):
    '''
        去除多余空白、分词、词性标注
    '''
    sentence = re.sub(r'\s+', ' ', sentence)
    token_words = word_tokenize(sentence)  # 输入的是列表
    token_words = pos_tag(token_words)
    return token_words


def stem(token_words):
    '''
        词形归一化
    '''
    wordnet_lematizer = WordNetLemmatizer()  # 单词转换原型
    words_lematizer = []
    for word, tag in token_words:
        if tag.startswith('NN'):
            word_lematizer = wordnet_lematizer.lemmatize(word, pos='n')  # n代表名词
        elif tag.startswith('VB'):
            word_lematizer = wordnet_lematizer.lemmatize(word, pos='v')  # v代表动词
        elif tag.startswith('JJ'):
            word_lematizer = wordnet_lematizer.lemmatize(word, pos='a')  # a代表形容词
        elif tag.startswith('R'):
            word_lematizer = wordnet_lematizer.lemmatize(word, pos='r')  # r代表代词
        else:
            word_lematizer = wordnet_lematizer.lemmatize(word)
        words_lematizer.append(word_lematizer)
    return words_lematizer


sr = stopwords.words('english')


def delete_stopwords(token_words):
    '''
        去停用词
    '''
    cleaned_words = [word for word in token_words if word not in sr]
    return cleaned_words


def is_number(s):
    '''
        判断字符串是否为数字
    '''
    try:
        float(s)
        return True
    except ValueError:
        pass

    try:
        import unicodedata
        unicodedata.numeric(s)
        return True
    except (TypeError, ValueError):
        pass

    return False


characters = [' ', ',', '.', 'DBSCAN', ':', ';', '?', '(', ')', '[', ']', '&', '!', '*', '@', '#', '$', '%', '-', '...',
              '^', '{', '}']


def delete_characters(token_words):
    '''
        去除特殊字符、数字
    '''
    words_list = [word for word in token_words if word not in characters and not is_number(word)]
    return words_list


def to_lower(token_words):
    '''
        统一为小写
    '''
    words_lists = [x.lower() for x in token_words]
    return words_lists


def pre_process(text):
    '''
        文本预处理
    '''
    token_words = tokenize(text)
    token_words = stem(token_words)
    token_words = delete_stopwords(token_words)
    token_words = delete_characters(token_words)
    token_words = to_lower(token_words)
    return token_words

In [ ]:
from tqdm import tqdm
article=list(data_p['title'].fillna(''))
article.extend(list(data_p['abstract'].fillna('')))
content=list(set(article))
train_content = list(map(lambda x: pre_process(x), tqdm(content)))

In [ ]:
import collections
#tfidf match share
print('get tfidf match share:')

def get_weight(count, eps=100, min_count=2):
    if count < min_count:
        return 0
    else:
        return 1 / (count + eps)


def load_weight(data):
    words = [x for y in data for x in y.split()]
    counts = collections.Counter(words)
    weights = {word: get_weight(count) for word, count in counts.items()}
    del counts
    del words
    del data
    gc.collect()
    return weights


def tfidf_match_share(queries, titles, weights):
    ret = []
    for i in tqdm(range(len(queries))):
        q, t = str(queries[i]).split(), str(titles[i]).split()
        q1words = {}
        q2words = {}
        for word in q:
            q1words[word] = 1
        for word in t:
            q2words[word] = 1
        if len(q1words) == 0 or len(q2words) == 0:
            # The computer-generated chaff includes a few questions that are nothing but stopwords
            R = 0
        else:
            shared_weights = [weights.get(w, 0) for w in q1words.keys() if w in q2words] + \
                             [weights.get(w, 0) for w in q2words.keys() if w in q1words]
            total_weights = [weights.get(w, 0) for w in q1words] + [weights.get(w, 0) for w in q2words]

            R = np.sum(shared_weights) / np.sum(total_weights)
        ret.append(R)
    return ret

In [ ]:
words = [x for y in train_content for x in y]
counts = collections.Counter(words)
del train_content,content
gc.collect()

weights = {word: get_weight(count) for word, count in counts.items()}
del counts
gc.collect()

In [ ]:
# f = open('weights3.txt','r')
# a = f.read()
# weights = eval(a)
# f.close()

In [ ]:
from tqdm import tqdm
data_p['tfidf_match_share'] = tfidf_match_share(data_p['key_text_pre'].values, data_p['title_pro'].values, weights)
data_p['tfidf_match_share_pa'] = tfidf_match_share(data_p['key_text_pre'].values, data_p['abstract_pre'].values, weights)

data_p['tfidf_match_share_2'] = tfidf_match_share(data_p['description_text'].values, data_p['title_pro'].values, weights)
data_p['tfidf_match_share_pa_2'] = tfidf_match_share(data_p['description_text'].values, data_p['abstract_pre'].values, weights)

In [ ]:
data_p['tfidf_match_share_3'] = tfidf_match_share(data_p['description_text'].values, data_p['title'].values, weights)
data_p['tfidf_match_share_pa_3'] = tfidf_match_share(data_p['description_text'].values, data_p['abstract'].values, weights)

In [ ]:
data_p.columns

In [ ]:
import lightgbm as lgb
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, auc

In [ ]:
le = preprocessing.LabelEncoder()
data_p['journal'] = le.fit_transform(data_p['journal'])

In [ ]:
data_p['paper_id_encode']=le.fit_transform(data_p['paper_id'])

In [ ]:
columns=['key_text_pre','abstract','abstract_pre','title','title_pro','description_text']
need_columns=[]
for col in columns:
    data_p[col+'+_len']=data_p[col].map(lambda x:len(str(x).split(' ')))
    need_columns.append(col+'+_len')

In [ ]:
zz=data_p.groupby('paper_id')['description_id'].agg('count').reset_index()
zz.columns=['paper_id','paper_id_count']
data_p=pd.merge(data_p,zz,on='paper_id',how='left')

In [ ]:
suanshu=['tfidf_cos','tfidf_os','tfidf_mhd','tfidf_cos2','tfidf_os2','tfidf_mhd2','tfidf_cos_2','tfidf_cos2_3','tfidf_os2_3','tfidf_mhd2_3','tfidf_cos2_4','tfidf_os2_4','tfidf_mhd2_4',
            'tfidf_os_2','tfidf_mhd_2','tfidf_cos2_2','tfidf_os2_2','tfidf_mhd2_2','score','tfidf_match_share','tfidf_match_share_pa','tfidf_match_share_2','tfidf_match_share_pa_2',
         'tfidf_match_share_3','tfidf_match_share_pa_3']
new_columns=[]

for i in range(len(suanshu)-1):
    coli=suanshu[i]
    for j in range(i+1,len(suanshu)):
        colj=suanshu[j]
        data_p[coli+'*'+colj]=data_p[coli]*data_p[colj]
        new_columns.append(coli+'*'+colj)

In [ ]:
mytrain=data_p[data_p.label.notnull()].reset_index(drop=True)
mytest=data_p[data_p.label.isnull()].reset_index(drop=True)

In [ ]:
mytrain=mytrain[mytrain.description_id!='6.45E+04'].reset_index(drop=True)

In [ ]:
mytrain=mytrain.sort_values(['description_id', 'score'], ascending=[True, False]).reset_index(drop=True)
mytest=mytest.sort_values(['description_id', 'score'], ascending=[True, False]).reset_index(drop=True)

In [ ]:
test_rank=[]
for i in range(len(mytest)//100):
    test_rank.extend([i+1 for i in range(100)])

In [ ]:
train_rank=[]
for i in range(len(mytrain)//100):
    train_rank.extend([i+1 for i in range(100)])

In [ ]:
mytrain['rank']=train_rank
mytest['rank']=test_rank

In [ ]:
data_p.columns,

In [ ]:
my_columns=['is_have_paper','tfidf_cos','tfidf_os','tfidf_mhd',
            'tfidf_cos2','tfidf_os2','tfidf_mhd2','tfidf_cos_2','tfidf_cos2_3',
            'tfidf_os2_3','tfidf_mhd2_3','tfidf_cos2_4','tfidf_os2_4','tfidf_mhd2_4',
            'tfidf_os_2','tfidf_mhd_2','tfidf_cos2_2','tfidf_os2_2','tfidf_mhd2_2',
            'score','rank','tfidf_match_share','tfidf_match_share_pa','tfidf_match_share_2',
            'tfidf_match_share_pa_2','tfidf_match_share_3','tfidf_match_share_pa_3',
            'journal','year','paper_id_encode','paper_id_count']+['key_text_pre+_len',
                                                 'abstract+_len',
                                                 'abstract_pre+_len',
                                                 'title+_len',
                                                 'title_pro+_len',
                                                 'description_text+_len']+new_columns

In [ ]:
train_x = mytrain[my_columns].values
train_y = mytrain['label'].values
test_x = mytest[my_columns].values

In [ ]:
lgb_clf = lgb.LGBMClassifier(num_leaves=32, 
                             learning_rate=0.1,
                             boosting='gbdt', 
                             max_depth=-1,
                             bagging_fraction=0.8,
                             n_estimators=40000,
                             bagging_freq=1,
                             feature_fraction=0.8, 
                             reg_alpha=0, 
                             reg_lambda=1,
                             lambda_l1=1,
                             lambda_l2=0.1,
                             metric='auc',
                             min_child_samples=5,
                             objective='binary')        


res = pd.DataFrame()
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5,shuffle=False, random_state=0)
baseloss = []
loss = 0
for i, (train_index, test_index) in enumerate(skf.split(train_x,train_y)):
    print("Fold", i)
    lgb_model = lgb_clf.fit(train_x[train_index], train_y[train_index],
                          eval_names =['train','valid'],
                          eval_metric='auc',
                          eval_set=[(train_x[train_index], train_y[train_index]),
                                    (train_x[test_index], train_y[test_index])],early_stopping_rounds=50)
    baseloss.append(lgb_model.best_score_['valid']['auc'])
    loss += lgb_model.best_score_['valid']['auc']
    test_pred= lgb_model.predict_proba(test_x, num_iteration=lgb_model.best_iteration_)[:, 1]
    print('test mean:', test_pred.mean())
    res['prob_%s' % str(i)] = test_pred
print('logloss:', baseloss, loss/5)

In [ ]:
res['prob'] = 0
for i in range(5):
    res['prob'] += res['prob_%s' % str(i)]
res['prob'] = res['prob'] / 5

In [ ]:
res['description_id'] = mytest['description_id']
res['paper_id'] = mytest['paper_id']

In [ ]:
res = res[['description_id', 'paper_id', 'prob']]

hh = res.sort_values(['description_id', 'prob'], ascending=[True, False])
hh = hh.reset_index(drop=True)

result = []
for i in range(0, len(hh), 100):
    line = hh.iloc[i:i + 3]
    result.append(list(line['paper_id']))

In [ ]:
result=pd.DataFrame(result)
result.columns=['paper'+str(i) for i in range(1,11)]

In [ ]:
press2 = pd.read_csv('./data/test_pre2.csv')
press2=press2.sort_values(by='description_id',ascending=True).reset_index(drop=True)

In [ ]:
result['description_id']=press2['description_id']

In [ ]:
result[['description_id','paper1','paper2','paper3']].to_csv('result/lgb_result2_4.csv',index=None,header=None)